In [ ]:
!pip install langchain
!pip install langchain-chroma
!pip install langchain_groq

### Documents

In [3]:
from langchain_core.documents import Document

documents = [
    Document(
    page_content="Dogs are great companions, know for their loyalty",
    metadata={"source": "mammal-pats-doc"}),
    Document(
    page_content="cats are independent pets that often enjoy their own space",
    metadata={"source": "mammal-pats-doc"}),
    Document(
    page_content="Goldfish are popular pets for beginners, requiring relatively simple care",
    metadata={"source": "fish-pats-doc"}),
    Document(
    page_content="Parrots are intelligent birds capable of mimicking human speech",
    metadata={"source": "bird-pats-doc"}),
    
]

In [4]:
documents

[Document(metadata={'source': 'mammal-pats-doc'}, page_content='Dogs are great companions, know for their loyalty'),
 Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'fish-pats-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
 Document(metadata={'source': 'bird-pats-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech')]

In [ ]:
pip install langchain_chroma

In [11]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key,model='Llama3-8b-8192')
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028C62D32AE0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028C62D33470>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
!pip install langchain_huggingface

In [17]:
# Embedding
from langchain_huggingface import HuggingFaceEmbeddings

model_kwargs = {'batch_size': 16}
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

d:\RAG-App\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\RAG-App\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\gupta\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this 

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [18]:
## VectorStore 
from langchain_chroma import Chroma 

vectorstore = Chroma.from_documents(documents,embedding=embeddings)
vectorstore


In [19]:
vectorstore.similarity_search('cat')

[Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pats-doc'}, page_content='Dogs are great companions, know for their loyalty'),
 Document(metadata={'source': 'bird-pats-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'fish-pats-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care')]

In [20]:
#Async query
await vectorstore.asimilarity_search('cat')

[Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pats-doc'}, page_content='Dogs are great companions, know for their loyalty'),
 Document(metadata={'source': 'bird-pats-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
 Document(metadata={'source': 'fish-pats-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care')]

In [21]:
vectorstore.similarity_search_with_score('cat')

[(Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space'),
  0.9436445236206055),
 (Document(metadata={'source': 'mammal-pats-doc'}, page_content='Dogs are great companions, know for their loyalty'),
  1.4742136001586914),
 (Document(metadata={'source': 'bird-pats-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech'),
  1.6284856796264648),
 (Document(metadata={'source': 'fish-pats-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care'),
  1.7496693134307861)]

### Retrievers


In [23]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["Cat","Shark"])

[[Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'fish-pats-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care')]]

vectorstore implement an as_retriever

In [31]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriever.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pats-doc'}, page_content='cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pats-doc'}, page_content='Dogs are great companions, know for their loyalty')]]

### Basic RAG

In [34]:
#chain

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.
{question}

context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human",message)])

rag_chain = {
    "context":retriever, "question":RunnablePassthrough()
}| prompt|llm


response = rag_chain.invoke("tell me about dogs")
response.content

'According to the provided context, dogs are great companions known for their loyalty.'